In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
path = '/global/homes/e/ecastori/PostBorn/'

In [ ]:
data = np.loadtxt(path+'FFT_k-1Pk.dat',skiprows=2)

#----------------------#
n         = data[:,0]
Re_c_n    = data[:,1]
Im_c_n    = data[:,2]
Re_nu_n   = data[:,3]
Im_nu_n   = data[:,4]
#----------------------#

c_n  = Re_c_n  + 1j * Im_c_n
nu_n = Re_nu_n + 1j * Im_nu_n

In [ ]:
data = np.loadtxt(path+'Il_nu_t.dat',skiprows=1)

#----------------------#
ell       = data[:,0]
nn        = data[:,2]
tt        = data[:,1]
Re_I      = data[:,3]
Im_I      = data[:,4]
#----------------------#

I_ = Re_I + 1j * Im_I

In [ ]:
#cosmology
h         = 0.6770
omega_cdm = 0.11923
omega_b   = 0.02247
Omega_b   = omega_b/h**2
Omega_cdm = omega_cdm/h**2
Omega_m   = Omega_b+Omega_cdm
ns        = 0.96824
A0        = 2.10732*10**(-9)

#dictionary for class
cosmo_dict={'h': h,
'omega_b' : omega_b,
'omega_cdm': omega_cdm,
'A_s'    : A0,
'n_s'    : ns,
'k_pivot' : 0.05}
print(cosmo_dict)

# speed of light
c      = 299792458./1000. # km/s

#prefactor for Cl_kk computation from Cl_dd
prefac = 1.5*Omega_m*(100.)**2/c**2 #without h

In [ ]:
from classy import Class
cosmo = Class()
cosmo.set(cosmo_dict)
cosmo.compute()
cosmo_b               = cosmo.get_background()

class_z               = cosmo_b['z'][::-1]
class_chi             = cosmo_b['comov. dist.'][::-1]
class_D               = cosmo_b['gr.fac. D'][::-1]/cosmo_b['gr.fac. D'][-1]

derivParams           = cosmo.get_current_derived_parameters(['z_rec'])
z_cmb                 = derivParams['z_rec']


from scipy.interpolate import InterpolatedUnivariateSpline as ius
chi_z = ius(class_z,class_chi*h)
z_chi = ius(class_chi*h,class_z)  # Mpc/h
D_chi = ius(class_chi*h,class_D)    # growth
D_z   = ius(class_z,class_D)
# chi_cmb
chi_cmb = chi_z(z_cmb)

In [ ]:
z=np.linspace(0.,z_cmb,200)
plt.figure()
plt.plot(z,chi_z(z))
plt.show()
plt.figure()
plt.plot(z,D_chi(chi_z(z)))
plt.show()
plt.figure()
plt.plot(z,z_chi(chi_z(z)))
plt.show()

In [ ]:
D_chi(chi_z(0.1))

In [ ]:
def set_kernel(chi_max):
    def kernel(chi):
        return (1.+z_chi(chi))*D_chi(chi)
    return kernel

In [ ]:
W = set_kernel(chi_cmb)

In [ ]:
ell_ = np.unique(ell)
nu_n_= np.unique(nu_n) 
print(nu_n_)
print(ell_)
print(np.unique(tt))
t_=np.unique(tt)
print(len(nu_n_),len(n),len(c_n),len(t_))
print(len(tt),len(I_))

In [ ]:
nu=nu_n_

res=[]
m = 0
for ii,ll in enumerate(ell_):
    print(ii,ll)
    for t in t_:
        for r in t_:
            res+=[np.real(sum(I_[m:m+len(n)][0:-1]*c_n[0:-1]*2.*(chi_cmb*r)**(-nu[0:-1]))+I_[m:m+len(n)][-1]*c_n[-1]*(chi_cmb*r)**(-nu[-1]))]
        m+=len(n)
assert(m==len(I_))



In [ ]:
m

In [ ]:
res=np.asarray(res)

In [ ]:
# m = 0
# r=0.02

# for ii,ll in enumerate(ell_):
#         print(ii,ll)
#         for jj,t in enumerate(t_[0:4]):
#             print('t',t)
#             print(np.real(sum(I_[ii*len(t_)+jj:ii*len(t_)+jj+len(n)][0:-1]*c_n[0:-1]*2.*(chi_cmb*r)**(-nu[0:-1]))+I_[m:m+len(n)][-1]*c_n[-1]*(chi_cmb*r)**(-nu[-1])))
#             m+=len(n)

In [ ]:
(chi_cmb*r)**(-nu[-1])

In [ ]:
np.real(c_n)

In [ ]:
print(np.real(res[0:100]))

In [ ]:
from scipy.integrate import simps, quadrature
from scipy.interpolate import interp1d
m=0
result=[]
for ii,ll in enumerate(ell_):
    res_=[]
    for t in t_:
        
        integrand=[]
        for r in t_:
            integrand+=[W(t*r*chi_cmb)*(1.-r*t)/t*(1.-r)/r*res[m]]

            
            m+=1
        integrand[0] = 0.+1j*0.
        #integrand = interp1d(t_,integrand,kind='cubic')
        res_+=[simps(integrand,t_)]#quadrature(integrand,min(t_),max(t_))[0]]#
    #print(res_)
        # for every n do r integral
    integrand = np.asarray(res_)*W(t_*chi_cmb)
    if ll==200. or ll==3.:
        plt.plot(t_,integrand,marker='o',ls='')
    
    integrand[0] = 0.+1j*0.
    #integrand = interp1d(t_,integrand,kind='cubic')
    result+=[simps(integrand,t_)]#min(t_),max(t_))[0]]
        

In [ ]:
Cl=np.real(np.asarray(result))*4./np.pi**2*prefac**2
print(Cl)

In [ ]:
plt.figure()
plt.loglog(ell_,1/4.*(ell_*(ell_+1.))**2*Cl)
plt.xlim(2,500)

In [ ]:
cosmo = Class()

cosmo_dict={'h': h,
'omega_b' : omega_b,
'omega_cdm': omega_cdm,
'A_s'    : A0,
'n_s'    : ns,
'k_pivot' : 0.05}
cosmo_dict['output']='lCl, tCl'
cosmo_dict['l_max_scalars']=2000
cosmo_dict['lensing']='yes'
cosmo_dict['l_switch_limber']=100

cosmo.set(cosmo_dict)

In [ ]:
cosmo.compute()

In [ ]:
cllen= cosmo.lensed_cl(lmax=1000)

In [ ]:
data=np.loadtxt(path+'Cell_marko.dat')
cl = data[:,1]

ls = data[:,0]
ll = cllen['ell']
clpp = cllen['pp']

In [ ]:
plt.figure()
plt.loglog(ell_,Cl)
plt.loglog(ll,clpp)
plt.loglog(ls,cl)

In [ ]:
Cl

In [ ]:
plt.figure()#
plt.semilogx(ell_,np.interp(ell_,ll,clpp)/Cl, label='our result')
#plt.semilogx(ell_,np.interp(ell_,ls,cl)/Cl)
plt.semilogx(ls,np.interp(ls,ll,clpp)/cl, label='Markos result')
#plt.axhline(1,color='black')
plt.ylim(0,2)
plt.grid()
plt.legend()